<a href="https://colab.research.google.com/github/danrosher/Algorithms/blob/master/JobQueryDocSemanticSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -qq install numpy datasets sentence_transformers transformers pandas

In [3]:

import numpy as np
from datasets import load_dataset

from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

from random import sample, seed, shuffle
from sentence_transformers import InputExample, losses, evaluation
from torch.utils.data import DataLoader
import pandas as pd

In [4]:
model_name='msmarco-MiniLM-L12-cos-v5'
#model_name='all-MiniLM-L6-v2'
#model_name='all-mpnet-base-v2'
#model_name='multi-qa-distilbert-cos-v1'
model = SentenceTransformer(model_name)
#sbert_model.max_seq_length = 256



.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.13k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [5]:
from google.colab import drive

drive.mount('/g')

Mounted at /g


In [6]:
df = pd.read_csv('/g/MyDrive/JobQueryDoc/docs/docs.csv')
df['c'] = df['c'].str[:800]

In [7]:
df.head()

,q,c,l
0,energy & manufacturing data engineer,Project Engineer - Offshore - Oil/Gas Industry...,1
1,scaffolders,Advanced Scaffolders Search Consultancy are lo...,1
2,part time sales,PART TIME SALES EXECUTIVE Our client is a UK w...,1
3,engineering project manager,Project Manager Imperial Professionals are del...,1
4,engineering project manager,Design Engineer Are you an experienced Design ...,1


In [8]:
def convert_data(df):
  return list(df.apply(lambda x: InputExample(texts = (x['q'], x['c']),label = float(x['l'])), axis = 1))

In [33]:
good_training_data = df[df['l']==1].sample(n=2000)
bad_training_data = df[df['l']==0].sample(n=2000)

In [34]:
#training_data = df.sample(frac=1) #return all in random order
sampled_training_data = convert_data(pd.concat([good_training_data,bad_training_data]).sample(frac=1))
len(sampled_training_data)

4000

In [35]:
sampled_training_data[0].__dict__

{'guid': '',
 'texts': ('pcb design engineer',
  'Carpenter Multi Trader Ignite Recruitment Services have over 20 years of experience in the construction industry. We strongly believe in â\x80\x9cbeing of serviceâ\x80\x9d to our clients. We make it a point to ensure that the experience candidates and clients have with us is only a positive one.\nThe construction industry is not a 9-5 job; and neither do we subscribe to this mind set. We believe that a construction recruitment company needs to be always available on the phone or in person 24/7/365.\nWE ARE CURRENTLY LOOKING FOR AN MULTI TRADE CARPENTER\nRequirements:\nSocial housing experience is required\nYou MUST have a full clean driving licence and ENHANCED DBS CHECK\nCarpenter multi required to work in responsive repairs dept with knowledge and experience working in social housing sector. Company van a'),
 'label': 0.0}

In [36]:
# Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(sampled_training_data, shuffle=True, batch_size=64)

train_loss = losses.CosineSimilarityLoss(model)

In [37]:
training_index = int(.8 * len(sampled_training_data))  # Get an 80/20 train/test split
train_examples = sampled_training_data[:training_index]


In [38]:
# Evaluation data
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(sampled_training_data[training_index:], main_similarity=evaluation.SimilarityFunction.COSINE)
model.evaluate(evaluator)

0.675208617524212

In [39]:
# Tune the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    output_path='/g/MyDrive/JobQueryDoc/model',
    save_best_model=True,
    epochs=10,
#     warmup_steps=len(sampled_training_data) // 5,
    evaluator=evaluator,
    evaluation_steps=100
)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

In [40]:
evaluator(model,output_path='/g/MyDrive/JobQueryDoc/model')

0.7630695927935439

In [41]:
model.save('/g/MyDrive/JobQueryDoc/model')


Now we create a **student** model  [see here](https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/distillation/dimensionality_reduction.py)

In [42]:
training_data = df[df['l']==1].sample(n=20000)


In [43]:
training_data

,q,c,l
19074,complaints,"Complaints Executive Our client, a leading glo...",1
15672,procurement,Buyer Pharmaceuticals Job description\n\nWe lo...,1
16592,lgv driver c1,Driver LGV (C1) Competitive salary + bonus\n\n...,1
8403,it tester,System Test Analyst Join a Thriving Team as a ...,1
8925,warehouse,Warehouse Operative/Loader Unloader AMJ Recrui...,1
...,...,...,...
11181,business intelligence,Tableau BI Developer Are you a skilled and pas...,1
10044,healthcare assistant,Community Care Assistant Are you a Community C...,1
26425,catering assistant,Catering Assistant required - Gravesend Cateri...,1
9544,administrator,Buying Manager Administrator Discovery is at t...,1


In [44]:
new_dimension  = 32

In [45]:
model = SentenceTransformer('/g/MyDrive/MyDrive/JobQueryDoc/model')


In [46]:
train_embeddings = model.encode(training_data['c'].values.tolist(), convert_to_numpy=True)


In [47]:
#Compute PCA on the train embeddings matrix
from sklearn.decomposition import PCA
import torch


pca = PCA(n_components=new_dimension)
pca.fit(train_embeddings)
pca_comp = np.asarray(pca.components_)

In [48]:
# We add a dense layer to the model, so that it will produce directly embeddings with the new size
from sentence_transformers import models

dense = models.Dense(in_features=model.get_sentence_embedding_dimension(), out_features=new_dimension, bias=False, activation_function=torch.nn.Identity())
dense.linear.weight = torch.nn.Parameter(torch.tensor(pca_comp))
model.add_module('dense', dense)

In [49]:
# Evaluate the model with the reduce embedding size
evaluator(model)

0.7646828134115513

In [50]:
model.save('/g/MyDrive/JobQueryDoc/model_student')
